# 뉴스 기사 크롤링 후 뉴스 챗봇



#크롤링 

In [1]:

import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime
import re


def preprocess_text(text):
    """한글 텍스트 전처리 (개선된 버전)"""
    if text == "":
        return ""

    # 특수문자 제거 (한글, 영어, 숫자, 공백만 유지)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', '', str(text))

    # '...' (마침표 3개)를 공백으로 대체
    text = re.sub(r'\.{3,}', ' ', text)

    # 2개 이상의 연속된 공백을 하나의 공백으로 변경
    text = re.sub(r'\s{2,}', ' ', text)

    return text


def crawl_naver_news():
    """네이버 뉴스 메인 페이지에서 헤드라인을 크롤링하는 함수"""

    print("📰 네이버 뉴스 크롤링을 시작합니다...")

    # 웹 페이지 URL
    url = "https://news.naver.com"

    # 헤더 설정 (코랩 환경에 맞게 조정)
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    try:
        # 웹 페이지 요청
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # HTML 파싱
        soup = BeautifulSoup(response.text, 'html.parser')

        # 뉴스 헤드라인 추출
        headlines = []

        # 여러 가지 방법으로 뉴스 헤드라인 찾기
        selectors = [
            'a.cjs_news_link',
            'a[href*="/article/"]',
            '.hdline_article_tit',
            '.cluster_text_headline'
        ]

        for selector in selectors:
            items = soup.select(selector)
            if items:
                for item in items[:-1]:  # 더 많은 헤드라인 수집
                    title = item.get_text().strip()
                    if title and len(title) > 10:
                        title = preprocess_text(title)
                        headlines.append(title)
                break

        # 결과 출력
        print(f"\n 총 {len(headlines)}개의 뉴스 헤드라인을 찾았습니다!")
        print("=" * 60)

        for i, headline in enumerate(headlines[:10], 1):
            print(f"{i:2d}. {headline}")

        print("=" * 60)

        return headlines

    except requests.RequestException as e:
        print(f" 네트워크 오류: {e}")
        return []
    except Exception as e:
        print(f" 크롤링 오류: {e}")
        return []
    


def create_news_dataframe(headlines):
    """뉴스 헤드라인을 데이터프레임으로 변환"""

    if not headlines:
        print(" 분석할 헤드라인이 없습니다.")
        return None

    # 데이터프레임 생성
    df = pd.DataFrame({
        '순번': range(1, len(headlines) + 1),
        '헤드라인': headlines,
        '글자수': [len(headline) for headline in headlines],
        '크롤링시간': [datetime.now().strftime('%Y-%m-%d %H:%M:%S')] * len(headlines)
    })

    print("\n 뉴스 데이터 분석:")
    print(f"총 뉴스 개수: {len(df)}")
    print(f"평균 글자수: {df['글자수'].mean():.1f}")
    print(f"최대 글자수: {df['글자수'].max()}")
    print(f"최소 글자수: {df['글자수'].min()}")

    return df

headlines = crawl_naver_news()
df = create_news_dataframe(headlines)

# 파일 저장
if df is not None:
    # CSV 파일로 저장
    df.to_csv('news_headlines.csv', index=False, encoding='utf-8')
    print("\n 'news_headlines.csv' 파일로 저장되었습니다!")

📰 네이버 뉴스 크롤링을 시작합니다...

 총 427개의 뉴스 헤드라인을 찾았습니다!
 1. 코스피 081 내린 315156에 마감 한국경제TV 내용작성전
 2. 코스피 2572포인트내린 315156 마감 머니S 내용작성전
 3. 정성호 법무장관 건진법사 관봉권 띠지 분실 감찰 지시 매경이코노미 내용작성전
 4. 이준석 김건희 특검 출석포렌식 참여 JTBC 내용작성전
 5. 한수원웨스팅하우스 계약 논란에 대통령실 산업부 진상 보고 지시 CJB청주방송
 6. 대검 건진법사 관봉권 띠지 분실 감찰 착수 KBS 내용작성전
 7. 정성호 법무 건진 돈다발 띠지 분실한 남부지검 감찰 등 지시
 8. 대통령실 불공정 계약 논란 체코 원전 수출 진상 파악 지시
 9. 김의겸 청담동 술자리 의혹 한동훈 손해배상 패소에 항소
10. 부부 계엄 책임 소송 시민들 서초동 자택 가압류 신청

 뉴스 데이터 분석:
총 뉴스 개수: 427
평균 글자수: 34.1
최대 글자수: 89
최소 글자수: 15

 'news_headlines.csv' 파일로 저장되었습니다!


# Chat Bot

In [ ]:
!pip install sentence_transformers


In [1]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head(50)

train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)


def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

def return_answer(question):
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']

/Users/kwangsiklee/miniforge3/envs/hf_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
return_answer('너는 누구니?')



'저는 위로봇입니다.'